In [1]:
!pip install gymnasium renderlab

Defaulting to user installation because normal site-packages is not writeable


In [2]:
!pip install opencv-python

Defaulting to user installation because normal site-packages is not writeable


In [3]:
!pip install pygame

Defaulting to user installation because normal site-packages is not writeable


In [4]:
import gymnasium as gym

In [5]:
#visualise maze:

desc=["SFFF", "FHHH", "FFFF", "HFHF", "FFGF"]

env = gym.make('FrozenLake-v1', desc=desc, map_name="5x5", is_slippery=False, render_mode="human") 

In [6]:
observation, info = env.reset()

In [12]:
env.close()